### 금융 문서 data에서 초해상화 학습을 위한 데이터 생성
  - input: 다양한 size, 비율의 서류 이미지
  - output: 같은 A4 비율로 crop한 hr: 1280,896 / lr 320,224 이미지

### onedrive mounting

In [ ]:
!wget https://downloads.rclone.org/v1.57.0/rclone-v1.57.0-linux-amd64.deb
!apt install ./rclone-v1.57.0-linux-amd64.deb

--2022-10-14 13:33:09--  https://downloads.rclone.org/v1.57.0/rclone-v1.57.0-linux-amd64.deb
Resolving downloads.rclone.org (downloads.rclone.org)... 95.217.6.16, 2a01:4f9:c012:7154::1
Connecting to downloads.rclone.org (downloads.rclone.org)|95.217.6.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14610238 (14M) [application/vnd.debian.binary-package]
Saving to: ‘rclone-v1.57.0-linux-amd64.deb’

rclone-v1.57.0-linu 100%[===================>]  13.93M  8.06MB/s    in 1.7s    

2022-10-14 13:33:12 (8.06 MB/s) - ‘rclone-v1.57.0-linux-amd64.deb’ saved [14610238/14610238]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'rclone' instead of './rclone-v1.57.0-linux-amd64.deb'
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  rclone
0 upgraded, 1 newly installed, 0 to rem

In [ ]:
!rclone config

2022/10/14 13:34:29 NOTICE: Config file "/root/.config/rclone/rclone.conf" not found - using defaults
No remotes found - make a new one
n) New remote
s) Set configuration password
q) Quit config
n/s/q> n
name> onedrive
Option Storage.
Type of storage to configure.
Enter a string value. Press Enter for the default ("").
Choose a number from below, or type in your own value.
 1 / 1Fichier
   \ "fichier"
 2 / Alias for an existing remote
   \ "alias"
 3 / Amazon Drive
   \ "amazon cloud drive"
 4 / Amazon S3 Compliant Storage Providers including AWS, Alibaba, Ceph, Digital Ocean, Dreamhost, IBM COS, Minio, SeaweedFS, and Tencent COS
   \ "s3"
 5 / Backblaze B2
   \ "b2"
 6 / Better checksums for other remotes
   \ "hasher"
 7 / Box
   \ "box"
 8 / Cache a remote
   \ "cache"
 9 / Citrix Sharefile
   \ "sharefile"
10 / Compress a remote
   \ "compress"
11 / Dropbox
   \ "dropbox"
12 / Encrypt/Decrypt a remote
   \ "crypt"
13 / Enterprise File Fabric
   \ "filefabric"
14 / FTP Connection
  

In [ ]:
!mkdir onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: ./onedrive &    

mkdir: cannot create directory ‘onedrive’: File exists
nohup: appending output to 'nohup.out'


### google drive mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### train_downscaling

In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import math

In [ ]:
### 224 * 320
IMG_WIDTH_LR = 224
IMG_HEIGHT_LR = 320
IMG_WIDTH_HR = 896
IMG_HEIGHT_HR = 1280

In [ ]:
dir_insure = '/content/onedrive/playdata/final_project/train/insure/images'
dir_insure_lr = '/content/drive/MyDrive/final_project/colab/data/insure_images_lr'
dir_insure_hr = '/content/drive/MyDrive/final_project/colab/data/insure_images_hr'

In [ ]:
file_list = os.listdir(dir_insure)
s_file_list = set(os.listdir(dir_insure_lr))

In [ ]:
for filename in file_list:
  if filename not in s_file_list:
    img = cv2.imread(f'{dir_insure}/{filename}', cv2.IMREAD_GRAYSCALE)
    img_hr = cv2.resize(img, (int(img.shape[1]/(img.shape[0]/IMG_HEIGHT_HR)),IMG_HEIGHT_HR), interpolation=cv2.INTER_AREA)
    temp = img_hr.shape[1] - IMG_WIDTH_HR
    margin = [np.abs(temp) // 2, np.abs(temp) // 2]
    if np.abs(temp) % 2 != 0:
        margin[0] += 1
    if temp > 0:
      if temp == 1:
        img_hr = img_hr[:,1:]
      else:
        img_hr = img_hr[:,margin[0]:-margin[1]]
    elif temp < 0:
      img_hr = np.pad(img_hr, ((0,0),(margin[0], margin[1])), mode='constant', constant_values=255)

    img_lr = cv2.resize(img, (int(img.shape[1]/(img.shape[0]/IMG_HEIGHT_LR)),IMG_HEIGHT_LR), interpolation=cv2.INTER_AREA)

    temp = img_lr.shape[1] - IMG_WIDTH_LR
    margin = [np.abs(temp) // 2, np.abs(temp) // 2]
    if np.abs(temp) % 2 != 0:
        margin[0] += 1
    if temp > 0:
      if temp == 1:
        img_lr = img_lr[:,1:]
      else:
        img_lr = img_lr[:,margin[0]:-margin[1]]
    elif temp < 0:
      img_lr = np.pad(img_lr, ((0,0),(margin[0], margin[1])), mode='constant', constant_values=255)
    # print(img_lr.shape)
    # print(img_hr.shape)
    cv2.imwrite(f'{dir_insure_lr}/{filename}', img_lr)
    cv2.imwrite(f'{dir_insure_hr}/{filename}', img_hr)

In [ ]:
dir_bank = '/content/onedrive/playdata/final_project/train/bank/images'
dir_bank_lr = '/content/drive/MyDrive/final_project/colab/data/bank_images_lr'
dir_bank_hr = '/content/drive/MyDrive/final_project/colab/data/bank_images_hr'





In [ ]:
file_list = os.listdir(dir_bank)
s_file_list = set(os.listdir(dir_bank_lr))

In [ ]:
for filename in file_list:
  if filename not in s_file_list:
    img = cv2.imread(f'{dir_bank}/{filename}', cv2.IMREAD_GRAYSCALE)
    img_hr = cv2.resize(img, (int(img.shape[1]/(img.shape[0]/IMG_HEIGHT_HR)),IMG_HEIGHT_HR), interpolation=cv2.INTER_AREA)
    temp = img_hr.shape[1] - IMG_WIDTH_HR
    margin = [np.abs(temp) // 2, np.abs(temp) // 2]
    if np.abs(temp) % 2 != 0:
        margin[0] += 1
    if temp > 0:
      if temp == 1:
        img_hr = img_hr[:,1:]
      else:
        img_hr = img_hr[:,margin[0]:-margin[1]]
    elif temp < 0:
      img_hr = np.pad(img_hr, ((0,0),(margin[0], margin[1])), mode='constant', constant_values=255)

    img_lr = cv2.resize(img, (int(img.shape[1]/(img.shape[0]/IMG_HEIGHT_LR)),IMG_HEIGHT_LR), interpolation=cv2.INTER_AREA)

    temp = img_lr.shape[1] - IMG_WIDTH_LR
    margin = [np.abs(temp) // 2, np.abs(temp) // 2]
    if np.abs(temp) % 2 != 0:
        margin[0] += 1
    if temp > 0:
      if temp == 1:
        img_lr = img_lr[:,1:]
      else:
        img_lr = img_lr[:,margin[0]:-margin[1]]
    elif temp < 0:
      img_lr = np.pad(img_lr, ((0,0),(margin[0], margin[1])), mode='constant', constant_values=255)
    # print(img_lr.shape)
    # print(img_hr.shape)
    cv2.imwrite(f'{dir_bank_lr}/{filename}', img_lr)
    cv2.imwrite(f'{dir_bank_hr}/{filename}', img_hr)

### validation_downscaling

In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import math

In [ ]:
### 224 * 320
IMG_WIDTH_LR = 224
IMG_HEIGHT_LR = 320
IMG_WIDTH_HR = 896
IMG_HEIGHT_HR = 1280

In [ ]:
val_raw = '/content/drive/MyDrive/final_project/colab/data/validation'
val_lr = '/content/drive/MyDrive/final_project/colab/data/validation_lr'
val_hr = '/content/drive/MyDrive/final_project/colab/data/validation_hr'
data_path = '/content/drive/MyDrive/final_project/colab/data'

file_list = os.listdir(val_raw)

In [ ]:
lr = []
hr = []
for filename in file_list:
  # if filename not in s_file_list:
  img = cv2.imread(f'{val_raw}/{filename}', cv2.IMREAD_GRAYSCALE)
  img_hr = cv2.resize(img, (int(img.shape[1]/(img.shape[0]/IMG_HEIGHT_HR)),IMG_HEIGHT_HR), interpolation=cv2.INTER_AREA)
  temp = img_hr.shape[1] - IMG_WIDTH_HR
  margin = [np.abs(temp) // 2, np.abs(temp) // 2]
  if np.abs(temp) % 2 != 0:
      margin[0] += 1
  if temp > 0:
    if temp == 1:
      img_hr = img_hr[:,1:]
    else:
      img_hr = img_hr[:,margin[0]:-margin[1]]
  elif temp < 0:
    img_hr = np.pad(img_hr, ((0,0),(margin[0], margin[1])), mode='constant', constant_values=255)

  img_lr = cv2.resize(img, (int(img.shape[1]/(img.shape[0]/IMG_HEIGHT_LR)),IMG_HEIGHT_LR), interpolation=cv2.INTER_AREA)
  temp = img_lr.shape[1] - IMG_WIDTH_LR
  margin = [np.abs(temp) // 2, np.abs(temp) // 2]
  if np.abs(temp) % 2 != 0:
      margin[0] += 1
  if temp > 0:
    if temp == 1:
      img_lr = img_lr[:,1:]
    else:
      img_lr = img_lr[:,margin[0]:-margin[1]]
  elif temp < 0:
    img_lr = np.pad(img_lr, ((0,0),(margin[0], margin[1])), mode='constant', constant_values=255)
  print(img_lr.shape)
  print(img_hr.shape)
  temp1 = os.path.join(val_lr, filename)
  temp2 = os.path.join(val_hr, filename)
  cv2.imwrite(temp1, img_lr)
  cv2.imwrite(temp2, img_hr)
  lr.append(temp1)
  hr.append(temp2)


In [ ]:
import pickle

# val_df = pd.DataFrame({'lr':lr,
                        #  'hr':hr})

with open(r'/content/drive/MyDrive/final_project/colab/data/val_df.pkl', 'wb' ) as file:
    pickle.dump(val_df, file)

### image visualization

In [ ]:
img = cv2.imread('/content/drive/MyDrive/final_project/colab/data/financial_OCR/insure_00004.jpeg', cv2.IMREAD_GRAYSCALE)
print(img.shape)
img = cv2.resize(img, (int(img.shape[1]/(img.shape[0]/IMG_HEIGHT_HR)),IMG_HEIGHT_HR), interpolation=cv2.INTER_AREA)
print(img.shape)
temp = img.shape[1] - IMG_WIDTH_HR
print(temp)
img = img[:,temp//2:-(temp//2)]
print(img.shape)
plt.figure(figsize=(10,15))
plt.imshow(img, cmap='gray')

In [ ]:
img = cv2.imread('/content/drive/MyDrive/final_project/colab/data/financial_OCR/insure_00004.jpeg', cv2.IMREAD_GRAYSCALE)
print(img.shape)
img = cv2.resize(img, (int(img.shape[1]/(img.shape[0]/IMG_HEIGHT_LR)),IMG_HEIGHT_LR), interpolation=cv2.INTER_AREA)
print(img.shape)
temp = img.shape[1] - IMG_WIDTH_LR
print(temp)
img = img[:,temp//2:-(temp//2)]
print(img.shape)
plt.figure(figsize=(10,15))
plt.imshow(img, cmap='gray')

### make_train_df

In [ ]:
dir_insure_lr = '/content/drive/MyDrive/final_project/colab/data/insure_images_lr'
dir_insure_hr = '/content/drive/MyDrive/final_project/colab/data/insure_images_hr'

dir_bank_lr = '/content/drive/MyDrive/final_project/colab/data/bank_images_lr'
dir_bank_hr = '/content/drive/MyDrive/final_project/colab/data/bank_images_hr'

data_path = '/content/drive/MyDrive/final_project/colab/data'

lr_insure_files = os.listdir(dir_insure_lr)
hr_insure_files = set(os.listdir(dir_insure_hr))

lr_bank_files = os.listdir(dir_insure_lr)
hr_bank_files = set(os.listdir(dir_insure_hr))

In [ ]:
lr = []
hr = []

for file_name in lr_insure_files:
  if file_name in hr_insure_files:
    lr.append(os.path.join(data_path, 'insure_images_lr', filename))
    hr.append(os.path.join(data_path, 'insure_images_hr', filename))
  else:
    print('미포함')

for file_name in lr_bank_files:
  if file_name in hr_bank_files:
    lr.append(os.path.join(data_path, 'bank_images_lr', filename))
    hr.append(os.path.join(data_path, 'bank_images_hr', filename))
  else:
    print('미포함')

train_df = pd.DataFrame({'lr':lr,
                         'hr':hr})

In [ ]:
with open(os.path.join(data_path, 'train_df'), 'wb' ) as file:
    pickle.dump(train_df, file)

In [ ]:
with open('/content/drive/MyDrive/final_project/colab/data/train_df', 'rb') as f:
  data = pickle.load(f)

### make_validation_df

In [ ]:
val_lr = '/content/drive/MyDrive/final_project/colab/data/validation_lr'
val_hr = '/content/drive/MyDrive/final_project/colab/data/validation_hr'

data_path = '/content/drive/MyDrive/final_project/colab/data'

lr_files = os.listdir(val_lr)
hr_files = set(os.listdir(val_hr))

In [ ]:
lr = []
hr = []

for file_name in lr_files:
  if file_name in hr_files:
    lr.append(os.path.join(data_path, 'validation_lr', filename))
    hr.append(os.path.join(data_path, 'validation_hr', filename))
  else:
    print('미포함')

val_df = pd.DataFrame({'lr':lr,
                         'hr':hr})

In [ ]:
with open(os.path.join(data_path, 'val_df'), 'wb' ) as file:
    pickle.dump(val_df, file)